In [2]:
import numpy as np

# Transition and emission matrices, and initial probabilities
A = np.array([[0.6, 0.2, 0.2], [0.5, 0.3, 0.2], [0.4, 0.1, 0.5]])
B = np.array([[0.7, 0.1, 0.2], [0.1, 0.6, 0.3], [0.3, 0.3, 0.4]])
init = np.array([0.5, 0.2, 0.3])

# Observation sequence indices (corresponding to "up", "down", "unchanged")
obs_index = [0, 0, 2, 1, 2, 1, 0] 

# Number of states and length of the observation sequence
N = len(init)
T = len(obs_index)

# Forward algorithm implementation
alpha = np.zeros((N, T))

# Initialization step
for i in range(N):
    alpha[i, 0] = init[i] * B[i, obs_index[0]]

# Recursion step
for t in range(1, T):
    for j in range(N):
        alpha[j, t] = sum(alpha[i, t-1] * A[i, j] for i in range(N)) * B[j, obs_index[t]]

# Termination step: Calculate the total probability of the observation sequence
total_probability = sum(alpha[i, T-1] for i in range(N))

# Output the result
print("Forward Probabilities (alpha):")
print(alpha)
print("\nTotal Probability of the Observation Sequence:", total_probability)

Forward Probabilities (alpha):
[[3.50000000e-01 1.79200000e-01 2.52100000e-02 3.02824000e-03
  1.47273120e-03 2.11233200e-04 3.69826262e-04]
 [2.00000000e-02 8.50000000e-03 1.25880000e-02 6.62040000e-03
  9.45268800e-04 4.60284960e-04 2.23192336e-05]
 [9.00000000e-02 3.57000000e-02 2.21560000e-02 5.59128000e-03
  1.89014720e-03 4.28602080e-04 1.04581402e-04]]

Total Probability of the Observation Sequence: 0.0004967268975999999


In [3]:
import numpy as np

def viterbi_algorithm(obs, states, start_p, trans_p, emit_p):
    # Initialize tables
    max_prob = np.zeros((len(states), len(obs)))
    path = np.zeros((len(states), len(obs)), dtype=int)

    # Initialization step
    for i in range(len(states)):
        max_prob[i, 0] = start_p[i] * emit_p[i, obs[0]]
        path[i, 0] = i

    # Recursion step
    for t in range(1, len(obs)):
        new_path = np.zeros((len(states), len(obs)), dtype=int)
        for j in range(len(states)):
            (prob, state) = max(
                (max_prob[i, t - 1] * trans_p[i, j] * emit_p[j, obs[t]], i)
                for i in range(len(states))
            )
            max_prob[j, t] = prob
            new_path[j] = path[state]
            new_path[j, t] = j
        path = new_path

    # Termination step
    (prob, state) = max((max_prob[i, len(obs) - 1], i) for i in range(len(states)))
    optimal_path = path[state]

    return optimal_path, prob

# Define parameters
states = [0, 1, 2]
obs = [0, 0, 2, 1, 2, 1, 0]  # Observation indices corresponding to "up", "down", "unchanged"
start_p = [0.5, 0.2, 0.3]
trans_p = np.array([[0.6, 0.2, 0.2], [0.5, 0.3, 0.2], [0.4, 0.1, 0.5]])
emit_p = np.array([[0.7, 0.1, 0.2], [0.1, 0.6, 0.3], [0.3, 0.3, 0.4]])

# Solve using Viterbi algorithm
optimal_sequence, max_probability = viterbi_algorithm(obs, states, start_p, trans_p, emit_p)

# Output results
print("Optimal State Sequence:", [s + 1 for s in optimal_sequence])  # Convert to 1-based index
print("Maximum Probability:", max_probability)

Optimal State Sequence: [np.int64(1), np.int64(1), np.int64(3), np.int64(3), np.int64(3), np.int64(3), np.int64(1)]
Maximum Probability: 1.48176e-05
